In [ ]:
import pandas as pd


In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

df = pd.read_csv("/content/IMDB Dataset.csv")  # Replace path if needed

# Initialize tools
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = text.lower()
    text = re.sub(r"<.*?>", " ", text)           # remove HTML
    text = re.sub(r"[^a-z\s]", " ", text)        # remove non-letters
    tokens = nltk.word_tokenize(text)            # tokenize
    tokens = [lemmatizer.lemmatize(t) for t in tokens if t not in stop_words]
    return " ".join(tokens)

df["clean_review"] = df["review"].apply(preprocess)
print(df.head())


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                              review sentiment  \
0  One of the other reviewers has mentioned that ...  positive   
1  A wonderful little production. <br /><br />The...  positive   
2  I thought this was a wonderful way to spend ti...  positive   
3  Basically there's a family where a little boy ...  negative   
4  Petter Mattei's "Love in the Time of Money" is...  positive   

                                        clean_review  
0  one reviewer mentioned watching oz episode hoo...  
1  wonderful little production filming technique ...  
2  thought wonderful way spend time hot summer we...  
3  basically family little boy jake think zombie ...  
4  petter mattei love time money visually stunnin...  


In [ ]:
df

,review,sentiment,clean_review
0,One of the other reviewers has mentioned that ...,positive,one reviewer mentioned watching oz episode hoo...
1,A wonderful little production. <br /><br />The...,positive,wonderful little production filming technique ...
2,I thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...
3,Basically there's a family where a little boy ...,negative,basically family little boy jake think zombie ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter mattei love time money visually stunnin...
...,...,...,...
49995,I thought this movie did a down right good job...,positive,thought movie right good job creative original...
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,bad plot bad dialogue bad acting idiotic direc...
49997,I am a Catholic taught in parochial elementary...,negative,catholic taught parochial elementary school nu...
49998,I'm going to have to disagree with the previou...,negative,going disagree previous comment side maltin on...


In [ ]:
all_words = " ".join(df["clean_review"]).split()
total_words = len(all_words)
unique_words = len(set(all_words))

print("Total words:", total_words)
print("Vocabulary size:", unique_words)


Total words: 5906540
Vocabulary size: 89812


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df["clean_review"])

one_hot = tokenizer.texts_to_matrix(df["clean_review"], mode="binary")
print(one_hot.shape)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
X_bow = cv.fit_transform(df["clean_review"])

print("Vocabulary (BOW) size:", len(cv.vocabulary_))

# Show how many times each word occurred
word_counts = pd.DataFrame(
    X_bow.toarray().sum(axis=0),
    index=cv.get_feature_names_out(),
    columns=["count"]
).sort_values(by="count", ascending=False)

print(word_counts.head(20))


In [ ]:
# Bi-gram model
cv2 = CountVectorizer(ngram_range=(2,2))
X_bi = cv2.fit_transform(df["clean_review"])
print("Bi-gram vocab size:", len(cv2.vocabulary_))

# Tri-gram model
cv3 = CountVectorizer(ngram_range=(3,3))
X_tri = cv3.fit_transform(df["clean_review"])
print("Tri-gram vocab size:", len(cv3.vocabulary_))


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(df["clean_review"])

print("TF-IDF vocab size:", len(tfidf.vocabulary_))

# Get IDF scores
idf_scores = dict(zip(tfidf.get_feature_names_out(), tfidf.idf_))

# Sort by lowest to highest idf
top_tfidf = sorted(idf_scores.items(), key=lambda x: x[1])
print(top_tfidf[:20])
